In [259]:
import openai
import pandas as pd
import numpy as np

In [260]:
openai.api_key = "sk-X2HFbojkXplIAcfYQ5X946Dq62vbAEug5s5yvshh"

### Create prompt

In [261]:
intro = "Below we classify tweets by their sentiment as either being Positive, Negative or Neutral\n"
ex1 = "Tweet: @VirginAmerica it's really aggressive to blast obnoxious 'entertainment' in your guests' faces &amp; they have little recourse\
\nSentiment: Negative\n"
ex2 = "Tweet: @united Just submitted my response on the link you sent.\
\nSentiment: Neutral\n"
ex3 = "Tweet: @VirginAmerica I love the hipster innovation. You are a feel good brand.\
\nSentiment: Positive\n"

#### [dummy call to the api]

In [262]:
test = "@VirginAmerica this is great news! \
America could start flights to Hawaii by end of year http://t.co/r8p2Zy3fe4 via @Pacificbiznews"

In [263]:
prompt = intro + ex1 + ex2 + ex3 + "Tweet: " + test + "\nSentiment:"
response = openai.Completion.create(engine="davinci",
                                    prompt=prompt,
                                    top_p=0.1,
                                    logprobs=3,
                                    max_tokens=1)
response['choices'][0]['text']

' Positive'

### Grab data and sample a proportion randomly

In [294]:
sample_fraction = 1
dataset_file = "data/0.0_min_confidence.csv"

train = pd.read_csv(dataset_file)
train = train.sample(frac=sample_fraction).reset_index(drop=True)
train

,airline_sentiment,text
0,negative,@AmericanAir We all did! Skipper's walk-around...
1,negative,@SouthwestAir I have a voucher w swa that expi...
2,neutral,@united Is a snowboard boot bag included in th...
3,neutral,@JetBlue plans to offer direct flights from #J...
4,negative,@SouthwestAir yes we did file a report and wer...
...,...,...
14635,positive,@USAirways - Huge props to Parizad at checkin ...
14636,neutral,"@AmericanAir oh, yeah. I guess those are two d..."
14637,neutral,@SouthwestAir think you need to follow me for ...
14638,negative,@united customer service is atrocious! You hav...


### Run inference using GPT-3 API

In [295]:
def model(tweet):
    """ Returns gtp-3 sentiment prediction from a tweet """
    prompt = intro + ex1 + ex2 + "Tweet: " + tweet + "\nSentiment:"
    response = openai.Completion.create(engine="davinci",
                                        prompt=prompt,
                                        temperature=0.05,
                                        logprobs=3,
                                        max_tokens=1)
    return pd.Series(
        dict(response['choices'][0]['logprobs']['top_logprobs'][0]))

In [296]:
def sigmoid(x):
    return 2 / (1 + np.exp(-x))

In [297]:
def evaluate(x):
    return x.idxmax().lower()[1:]

In [ ]:
probs = train['text'].apply(model).applymap(sigmoid)
probs['prediction'] = probs.apply(evaluate, axis=1)
result = train.merge(probs, left_index=True, right_index=True)

### Calculate the accuracy

In [ ]:
loss = result['airline_sentiment'] == result['prediction']
loss.agg('sum') / loss.size

#### [output the predicted labels to a csv]

In [ ]:
output = dataset.loc[:, dataset.columns.isin(['text', 'prediction'])]
output.to_csv("data/gtp_predictions.csv", index=False)